<a href="https://colab.research.google.com/github/aswit3/Analystics_Vidhya_Sentiment_Analysis_Medical/blob/master/ELMo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd

In [0]:
tr = pd.read_csv("drive/My Drive/datasets/Sentiment_Analysis_Analytics_vidhya_Medical/train_F3WbcTw.csv")
te = pd.read_csv("drive/My Drive/datasets/Sentiment_Analysis_Analytics_vidhya_Medical/test_tOlRoBf.csv")

In [33]:
tr.shape, te.shape

((5279, 4), (2924, 3))

In [34]:
tr.head(2)

,unique_hash,text,drug,sentiment
0,2e180be4c9214c1f5ab51fd8cc32bc80c9f612e0,Autoimmune diseases tend to come in clusters. ...,gilenya,2
1,9eba8f80e7e20f3a2f48685530748fbfa95943e4,I can completely understand why you’d want to ...,gilenya,2


In [35]:
te.head(2)

,unique_hash,text,drug
0,9e9a8166b84114aca147bf409f6f956635034c08,"256 (previously stable on natalizumab), with 5...",fingolimod
1,e747e6822c867571afe7b907b51f0f2ca67b0e1a,On fingolimod and have been since December 201...,fingolimod


In [0]:
del tr["unique_hash"]
del te["unique_hash"]

In [37]:
len(tr["drug"].value_counts()), len(te["drug"].value_counts())

(102, 95)

In [38]:
tr.shape, te.shape

((5279, 3), (2924, 2))

In [39]:
tr_sentiment = tr["sentiment"]
tr_sentiment.shape

(5279,)

In [0]:
del tr["sentiment"]

In [41]:
tr.shape, te.shape

((5279, 2), (2924, 2))

In [0]:
trte = tr.append(te, ignore_index=True)

In [43]:
trte.shape

(8203, 2)

In [44]:
len(trte["drug"].value_counts())

111

In [0]:
import re

clean_list = []

for i, unclean_text in enumerate(trte["text"]):
  clean_text = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', '', unclean_text)
  clean_list.append(clean_text)

In [0]:
trte["text"] = clean_list

In [0]:
import re, string, unicodedata
import inflect
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

In [48]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    return words

In [0]:
words_ls = []
for i, unclean_text in enumerate(trte["text"]):
  words = nltk.word_tokenize(unclean_text)
  words = normalize(words)
  words_ls.append(" ".join(words))

In [0]:
trte['text'] = words_ls

In [56]:
text_tokens =trte['text']
text_tokens.shape

(8203,)

In [0]:
import tensorflow as tf
import tensorflow_hub as hub

In [0]:
option = 2

if option == 1:
  url = "https://tfhub.dev/google/elmo/2"
  embed = hub.Module(url)
elif option == 2:
  url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
  embed = hub.Module(url)

In [0]:
from keras.layers import Input, Lambda, Dense

input_text = Input(shape=(1,), dtype=tf.string)

if option == 1:
  def ELMoEmbedding(x):
      return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]
  embedding = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
elif option == 2:
  def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)))
  embedding = Lambda(UniversalEmbedding, output_shape=(512, ))(input_text)

In [0]:
import tensorflow as tf
import keras.layers as layers
from keras.models import Model
from keras.datasets import imdb
from gensim.models import Word2Vec
from gensim.models import FastText
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Input,Embedding,Dense,Flatten
from sklearn.metrics import accuracy_score,classification_report
from sklearn.metrics import f1_score

In [67]:
from sklearn.model_selection import train_test_split

train_w2v = text_tokens[:5279]
test_w2v = text_tokens[5279:]


_, _, ytrain, yvalid = train_test_split(train_w2v, tr_sentiment,  
                                                          random_state=42, 
                                                          test_size=0.3)



print(train_w2v.shape, tr_sentiment.shape)
print(ytrain.shape, yvalid.shape)


(5279,) (5279,)
(3695,) (1584,)


In [0]:
epochs = 25
batch_size = 1024
loss = "sparse_categorical_crossentropy"
optimizer = "adam"
metrics = ["accuracy"]

from keras import models

callbacks = [EarlyStopping(monitor='val_loss', patience=2),
            ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

x = Dense(256,activation ="relu")(embedding)
preds = Dense(3,activation="softmax")(x)
model = Model(input_text,preds)

model.compile(loss=loss,optimizer=optimizer,metrics= metrics)
model.fit(train_w2v,tr_sentiment,epochs=epochs,batch_size=batch_size,callbacks=callbacks,validation_data=(train_w2v[:500],tr_sentiment[:500]))

W0725 19:39:40.584172 140383520290688 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0725 19:39:40.629830 140383520290688 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0725 19:39:40.657180 140383520290688 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3341: The name tf.log is deprecated. Please use tf.math.log instead.

W0725 19:39:40.754471 140383520290688 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, 

Train on 5279 samples, validate on 500 samples
Epoch 1/25


In [0]:
predictions = model.predict(xvalid_w2v)
predictions = predictions.argmax(axis = -1)
df = pd.DataFrame()
df['new_col'] = predictions
f1_score(yvalid, df['new_col'], average='micro')

0.7316919191919192

In [0]:
test_pred = model.predict(test_w2v)
test_pred = test_pred.argmax(axis = -1)

te_res = pd.read_csv("drive/My Drive/datasets/Sentiment_Analysis_Analytics_vidhya_Medical/test_tOlRoBf.csv")
te_res['sentiment'] = test_pred

submission = te_res[['unique_hash','sentiment']]
submission.to_csv('model_result.csv', index=False)